In [1]:
import pandas as pd
from decimal import Decimal

from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )

Carbon Version v2.0-beta3 (12/Dec/2022)


# Carbon Simulation - Demo 5-4

In this demo we investigate the **Fast Router performance** against test orders

Initialize a fast simulator

In [2]:
Sim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='exact', raiseonerror=True)
FastSim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='fast', raiseonerror=True)

Here we restrain the orders to just the top ones such that we can test the exact algo alongside

In [3]:
test_orders = pd.read_csv('256 orders with curr rate alternating.csv', dtype=float)
test_orders.rename(columns={'lowest-rate':'lowest_rate', 'highest-rate':'highest_rate', 'current-rate':'current_rate'}, inplace=True)
test_orders['one_on_lowest_rate'] = 1/test_orders.lowest_rate
test_orders['one_on_highest_rate'] = 1/test_orders.highest_rate
test_orders['one_on_current_rate'] = 1/test_orders.current_rate
test_orders['y_int'] = [test_orders.liquidity[i] * (test_orders.one_on_highest_rate[i] - test_orders.one_on_lowest_rate[i]) / (test_orders.one_on_current_rate[i] - test_orders.one_on_lowest_rate[i]) for i in test_orders.index]
test_orders['fix_trade'] = test_orders.y_int - test_orders.liquidity
test_orders['y_int_minus_fixtrade'] = test_orders.y_int - test_orders.fix_trade
test_orders

,id,liquidity,lowest_rate,highest_rate,current_rate,one_on_lowest_rate,one_on_highest_rate,one_on_current_rate,y_int,fix_trade,y_int_minus_fixtrade
0,0.0,254814732.0,1255.0,2255.0,1256.0,0.000797,0.000443,0.000796,1.419279e+11,1.416730e+11,254814732.0
1,1.0,253827078.0,1254.0,2254.0,2254.0,0.000797,0.000444,0.000444,2.538271e+08,0.000000e+00,253827078.0
2,2.0,252839424.0,1253.0,2253.0,1753.0,0.000798,0.000444,0.000570,3.934554e+08,1.406160e+08,252839424.0
3,3.0,251851770.0,1252.0,2252.0,1253.0,0.000799,0.000444,0.000798,1.401289e+11,1.398770e+11,251851770.0
4,4.0,6913578.0,1004.0,2004.0,1504.0,0.000996,0.000499,0.000665,1.037727e+07,3.463689e+06,6913578.0
...,...,...,...,...,...,...,...,...,...,...,...
251,251.0,250864116.0,1251.0,2251.0,2251.0,0.000799,0.000444,0.000444,2.508641e+08,2.980232e-08,250864116.0
252,252.0,5925924.0,1003.0,2003.0,1004.0,0.000997,0.000499,0.000996,2.970358e+09,2.964432e+09,5925924.0
253,253.0,4938270.0,1002.0,2002.0,2002.0,0.000998,0.000500,0.000500,4.938270e+06,0.000000e+00,4938270.0
254,254.0,3950616.0,1001.0,2001.0,1501.0,0.000999,0.000500,0.000666,5.926911e+06,1.976295e+06,3950616.0


In [4]:
test_orders['fix_trade'][0]

141673035996.16202

To translate this order input (effectively p_a and p_b) into a meaningful order in the simulator we need to transform them
The primary reason for this is that the current_rate (akin to p_marg) starts at the highest_rate (that has been denoted p_a) but the simulator takes the p_marg as being the p_start value in this pair

In [5]:
for i in test_orders.index:
    # Sim.add_strategy('ETH', Decimal(test_orders.y_int[i]), 1/Decimal(test_orders.highest_rate[i]), 1/Decimal(test_orders.lowest_rate[i]),  None, None, None)
    FastSim.add_strategy('ETH', Decimal(test_orders.y_int[i]), 1/Decimal(test_orders.highest_rate[i]), 1/Decimal(test_orders.lowest_rate[i]),  None, None, None)
    if test_orders.fix_trade[i] > 0:
        print(i, i*2)
        # Sim.amm_sells('ETH', amt=test_orders.fix_trade[i], execute=True, use_positions=[i*2])['trades']
        FastSim.amm_sells('ETH', amt=test_orders.fix_trade[i], execute=True, use_positions=[i*2])['trades']

0 0
2 4
3 6
4 8
6 12
7 14
9 18
12 24
13 26
15 30
16 32
18 36
19 38
21 42
22 44
24 48
25 50
27 54
30 60
31 62
33 66
34 68
36 72
37 74
38 76
39 78
40 80
42 84
43 86
45 90
46 92
48 96
49 98
51 102
52 104
54 108
57 114
58 116
60 120
61 122
63 126
64 128
65 130
66 132
67 134
69 138
70 140
72 144
73 146
75 150
76 152
78 156
79 158
81 162
82 164
84 168
85 170
87 174
88 176
89 178
90 180
93 186
94 188
96 192
97 194
99 198
100 200
102 204
103 206
105 210
106 212
108 216
109 218
111 222
112 224
114 228
115 230
117 234
118 236
119 238
120 240
121 242
123 246
124 248
126 252
127 254
129 258
130 260
132 264
133 266
135 270
138 276
139 278
141 282
142 284
144 288
145 290
147 294
148 296
150 300
151 302
153 306
154 308
156 312
157 314
159 318
160 320
162 324
163 326
164 328
165 330
166 332
168 336
169 338
171 342
172 344
174 348
175 350
177 354
178 356
179 358
180 360
181 362
183 366
184 368
186 372
187 374
189 378
192 384
193 386
195 390
196 392
198 396
199 398
200 400
201 402
202 404
204 408
205 41

In [6]:
FastSim.state()['trades']

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,True,None,1.416730e+11,ETH,8.416406e+07,USDC,ETHUSDC,0,1,0.000594,USDC per ETH
0,0,0,A,AMM sells 141673035996ETH buys 84164062USDC,True,True,None,1.416730e+11,ETH,8.416406e+07,USDC,ETHUSDC,[0],1,0.000594,USDC per ETH
0,1.0,1,0,route #4,False,True,None,1.406160e+08,ETH,6.865070e+04,USDC,ETHUSDC,4,1,0.000488,USDC per ETH
0,1,1,A,AMM sells 140615978ETH buys 68651USDC,True,True,None,1.406160e+08,ETH,6.865070e+04,USDC,ETHUSDC,[4],1,0.000488,USDC per ETH
0,2.0,2,0,route #6,False,True,None,1.398770e+11,ETH,8.325184e+07,USDC,ETHUSDC,6,1,0.000595,USDC per ETH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,173,173,A,AMM sells 2964432387ETH buys 2089743USDC,True,True,None,2.964432e+09,ETH,2.089743e+06,USDC,ETHUSDC,[504],1,0.000705,USDC per ETH
0,174.0,174,0,route #508,False,True,None,1.976295e+06,ETH,1.094480e+03,USDC,ETHUSDC,508,1,0.000554,USDC per ETH
0,174,174,A,AMM sells 1976295ETH buys 1094USDC,True,True,None,1.976295e+06,ETH,1.094480e+03,USDC,ETHUSDC,[508],1,0.000554,USDC per ETH
0,175.0,175,0,route #510,False,True,None,1.480000e+09,ETH,1.045652e+06,USDC,ETHUSDC,510,1,0.000707,USDC per ETH


We can see that when we input the order with flipped rates we get p_marg == p_start.

Unfortunately this means that the pricing is flipped relative to the order input

In [7]:
FastSim.state()['orders'].query('tkn=="ETH"')

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,ETH,1.419279e+11,254814732.0,ETH,False,0.000443,0.000797,0.000796,USDC per ETH,1
2,2,ETHUSDC,ETH,2.538271e+08,253827078.0,ETH,False,0.000444,0.000797,0.000444,USDC per ETH,3
4,4,ETHUSDC,ETH,3.934554e+08,252839424.0,ETH,False,0.000444,0.000798,0.000537,USDC per ETH,5
6,6,ETHUSDC,ETH,1.401289e+11,251851770.0,ETH,False,0.000444,0.000799,0.000798,USDC per ETH,7
8,8,ETHUSDC,ETH,1.037727e+07,6913578.0,ETH,False,0.000499,0.000996,0.000613,USDC per ETH,9
...,...,...,...,...,...,...,...,...,...,...,...,...
502,502,ETHUSDC,ETH,2.508641e+08,250864116.0,ETH,False,0.000444,0.000799,0.000444,USDC per ETH,503
504,504,ETHUSDC,ETH,2.970358e+09,5925924.0,ETH,False,0.000499,0.000997,0.000995,USDC per ETH,505
506,506,ETHUSDC,ETH,4.938270e+06,4938270.0,ETH,False,0.0005,0.000998,0.0005,USDC per ETH,507
508,508,ETHUSDC,ETH,5.926911e+06,3950616.0,ETH,False,0.0005,0.000999,0.000614,USDC per ETH,509


## Route by Source

#### AMM buys USDC

We can then do a similar thing for the fast router

In [8]:
FastSim.amm_buys('USDC',10000000, execute=False)['trades']  # route_trade_by_source

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,176.0,176,0,route #2,False,False,None,2.538271e+08,ETH,1.509775e+05,USDC,ETHUSDC,2,1,0.00064,USDC per ETH
0,176.1,176,1,route #502,False,False,None,2.508641e+08,ETH,1.494934e+05,USDC,ETHUSDC,502,1,0.00064,USDC per ETH
0,176.2,176,2,route #496,False,False,None,2.479012e+08,ETH,1.480039e+05,USDC,ETHUSDC,496,1,0.00064,USDC per ETH
0,176.3,176,3,route #20,False,False,None,2.449382e+08,ETH,1.465088e+05,USDC,ETHUSDC,20,1,0.00064,USDC per ETH
0,176.4,176,4,route #484,False,False,None,2.419752e+08,ETH,1.450081e+05,USDC,ETHUSDC,484,1,0.00064,USDC per ETH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,176.89,176,89,route #100,False,False,None,5.234566e+07,ETH,3.567871e+04,USDC,ETHUSDC,100,1,0.00064,USDC per ETH
0,176.90,176,90,route #130,False,False,None,1.906172e+08,ETH,1.301192e+05,USDC,ETHUSDC,130,1,0.00064,USDC per ETH
0,176.91,176,91,route #94,False,False,None,4.938270e+07,ETH,3.373204e+04,USDC,ETHUSDC,94,1,0.00064,USDC per ETH
0,176.92,176,92,route #374,False,False,None,1.426899e+08,ETH,9.246902e+04,USDC,ETHUSDC,374,1,0.00064,USDC per ETH


## Route by Target

#### AMM sells ETH

In [9]:
FastSim.amm_sells('ETH',15633055257, execute=False)['trades']  # route_trade_by_target

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,177.0,177,0,route #2,False,False,None,2.538271e+08,ETH,1.509775e+05,USDC,ETHUSDC,2,1,0.00064,USDC per ETH
0,177.1,177,1,route #502,False,False,None,2.508641e+08,ETH,1.494934e+05,USDC,ETHUSDC,502,1,0.00064,USDC per ETH
0,177.2,177,2,route #496,False,False,None,2.479012e+08,ETH,1.480039e+05,USDC,ETHUSDC,496,1,0.00064,USDC per ETH
0,177.3,177,3,route #20,False,False,None,2.449382e+08,ETH,1.465088e+05,USDC,ETHUSDC,20,1,0.00064,USDC per ETH
0,177.4,177,4,route #484,False,False,None,2.419752e+08,ETH,1.450081e+05,USDC,ETHUSDC,484,1,0.00064,USDC per ETH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,177.89,177,89,route #100,False,False,None,5.234566e+07,ETH,3.567871e+04,USDC,ETHUSDC,100,1,0.00064,USDC per ETH
0,177.90,177,90,route #130,False,False,None,1.906172e+08,ETH,1.301192e+05,USDC,ETHUSDC,130,1,0.00064,USDC per ETH
0,177.91,177,91,route #94,False,False,None,4.938270e+07,ETH,3.373204e+04,USDC,ETHUSDC,94,1,0.00064,USDC per ETH
0,177.92,177,92,route #374,False,False,None,1.426899e+08,ETH,9.246902e+04,USDC,ETHUSDC,374,1,0.00064,USDC per ETH
